### 밑에 넷플릭스 아이디랑 비밀번호 입력하시고 그냥 전체 실행 하시면 됩니다!

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pathlib
import time
import os
import random

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import re
import pandas as pd

from tqdm import tqdm
import numpy as np
import urllib.request

In [3]:
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
service = Service(executable_path=ChromeDriverManager().install())

chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko")

In [2]:
netflix = pd.read_csv('netflix_original_final.csv')

In [8]:
browser = webdriver.Chrome(service=service,options=chrome_options)

browser.implicitly_wait(10)
# browser.maximize_window()
url = f"https://www.netflix.com/browse"
browser.get(url)
# 로그인 
browser.find_element(By.CSS_SELECTOR,'input#id_userLoginId').send_keys('이메일id')
time.sleep(1.5)
browser.find_element(By.CSS_SELECTOR,'input#id_password').send_keys('비밀번호')
browser.find_element(By.CSS_SELECTOR,'input#static-login-cta').click()



url_dict = {}
for i,title in tqdm(enumerate(netflix['제목'])):
    if i==5: break
    url = f'https://www.netflix.com/search?q={title}'
    browser.get(url)
    time.sleep(0.4)
    try:
        title_object = browser.find_elements(By.CSS_SELECTOR,'a.watch-title')
        for object in title_object:
            if str(netflix[netflix['제목']==title].nfid.values[0]) in object.get_attribute('href'):
                url = object.find_element(By.CSS_SELECTOR,'div.title-boxart').get_attribute('style')
                url = re.search('"(.+?)"',url).group(1)
                url_dict[netflix[netflix['제목']==title].nfid.values[0]]=url
    except:
        print(f'not_found = {title}')

df = pd.DataFrame([url_dict.keys(),url_dict.values()]).T
df.columns=['nfid','poster']
df.to_csv('poster.csv',index=False)

5it [00:19,  3.82s/it]
